In [1]:
from datetime import datetime
import datetime
B=datetime.datetime.now()

### Importing Packages

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import VotingClassifier
import pyodbc
from sqlalchemy import create_engine
import urllib
from keras.models import Sequential
from keras.layers import Dense

ModuleNotFoundError: No module named 'pyodbc'

### Importing Data

In [3]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=localhost;Database=HDM;Trusted_Connection=yes;')
cursor = cnxn.cursor()
sql = "Select * from HistoricalDataMod"
df = pd.read_sql(sql, cnxn)
df['Failure'].replace("Yes",1, inplace=True)
df['Failure'].replace("No",0, inplace=True)
df1=df.copy()
#Import important Libraries
from sqlalchemy import create_engine
import urllib
params = urllib.parse.quote_plus(r'DRIVER={SQL Server};Server=localhost;DATABASE=HDM;Trusted_Connection=yes')
c_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine = create_engine(c_str)

In [4]:
print("Total number of observations in data         :", df1.iloc[:,-1].count())
print("Total number of observations for Failure     :", df1[df1.iloc[:,-1]!=0].iloc[:,-1].count())
print("Total number of observations for not-Failure :", df1.iloc[:,-1].count()-df1[df1.iloc[:,-1]!=0].iloc[:,-1].count())
if df1[df1.iloc[:,-1]!=0].iloc[:,-1].count() < df1.iloc[:,-1].count()-df1[df1.iloc[:,-1]!=0].iloc[:,-1].count() :
    print("Maximum accuracy of a Dummy Classifier       :",round((df1.iloc[:,-1].count()-df1[df1.iloc[:,-1]!=0].iloc[:,-1].count())/df1.iloc[:,-1].count(),4))
else :
    print("Maximum accuracy of a Dummy Classifier       :",round((df1[df1.iloc[:,-1]!=0].iloc[:,-1].count())/df1.iloc[:,-1].count(),4))

NameError: name 'df1' is not defined

# Data Pre-Processing is Mandatory

In [2]:
j=0
for mlmodel in range(70,91,5): 
    df=df1.copy()
    x=df.iloc[:,:-1]
    rows, columns = x.shape
    print("Number of feature columns in data-set :",columns)
    y=df.iloc[:,-1]
    tsize=(100-mlmodel)/100
    x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=tsize, random_state=10)
    print("Number of rows in training :", x_train.iloc[:,-1].count())
    print("Number of rows in testing  :", x_test.iloc[:,-1].count())

    ### Scalling the Data
    scaler = MinMaxScaler()
    x_trainscaled = scaler.fit_transform(x_train)
    x_testscaled = scaler.transform(x_test)
    x_test['Actual Value']=y_test

    ### KNN
    knnclas=KNeighborsClassifier()
    k_value={'n_neighbors':range(1,50)}
    grid=GridSearchCV(knnclas,param_grid=k_value)
    grid.fit(x_trainscaled,y_train)
    print("The best training score is : ",'%.4f' %grid.score(x_trainscaled,y_train))
    print("The best test score is : ",'%.4f' %grid.score(x_testscaled,y_test))
    print(grid.best_params_)
    knnclasp=grid.predict(x_testscaled)
    print(confusion_matrix(y_test,knnclasp))
    x_knn=x_test.copy()
    grid1=grid
    x_knn['Predicted Value']=grid1.predict(x_testscaled)
    x_knn['Model']='KNN'

   
     ### Logistic Regression
    lrg=LogisticRegression()
    value={'penalty':['l1','l2'], 'C':[0.1,0.5,0.75,1,2,5,10,15,20,30,50,75,100]}
    grid=GridSearchCV(lrg,param_grid=value)
    grid.fit(x_trainscaled,y_train)
    print("The best training score is : ",'%.4f' %grid.score(x_trainscaled,y_train))
    print("The best test score is : ",'%.4f' %grid.score(x_testscaled,y_test))
    print(grid.best_params_)
    lrgp=grid.predict(x_testscaled)
    print(confusion_matrix(y_test,lrgp))
    x_log=x_test.copy()
    grid2=grid
    x_log['Predicted Value']=grid2.predict(x_testscaled)
    x_log['Model']='Logistic'

    ### Linear Support Vector Machines
    svcg=LinearSVC()
    value={'C':[0.1,0.2,0.3,0.4,0.5,0.6,0.75,1,2,5,10,15,20,30,50,75,100]}
    grid=GridSearchCV(svcg,param_grid=value)
    grid.fit(x_trainscaled,y_train)
    print("The best training score is : ",'%.4f' %grid.score(x_trainscaled,y_train))
    print("The best test score is : ",'%.4f' %grid.score(x_testscaled,y_test))
    print(grid.best_params_)
    svcgp=grid.predict(x_testscaled)
    print(confusion_matrix(y_test,svcgp))
    x_lsvc=x_test.copy()
    grid3=grid
    x_lsvc['Predicted Value']=grid3.predict(x_testscaled)
    x_lsvc['Model']='Linear SVC'

    ### RBF Kernel
    svcg=SVC(kernel='rbf')
    value = {'C':[0.1,1,5,10,12,15,17,20,25], 'gamma':[0.01,0.05,0.1,0.25,0.5, 1,2,5,10,12,15,20]}
    grid=GridSearchCV(svcg,param_grid=value)
    grid.fit(x_trainscaled,y_train)
    print("The best training score is : ",'%.4f' %grid.score(x_trainscaled,y_train))
    print("The best test score is : ",'%.4f' %grid.score(x_testscaled,y_test))
    print(grid.best_params_)
    svcgp=grid.predict(x_testscaled)
    print(confusion_matrix(y_test,svcgp))
    grid4=grid
    x_rbf=x_test.copy()
    x_rbf['Predicted Value']=grid4.predict(x_testscaled)
    x_rbf['Model']='RBF'
    

    ### Polynomial Kernel
    svcg=SVC(kernel='poly')
    value = {'degree':[2,3,4,5],'C':[0.1,0.5,1,2,5,10], 'gamma':[0.01,0.05,0.1,0.25,0.5, 1, 5]}
    grid=GridSearchCV(svcg,param_grid=value, scoring='recall')
    grid.fit(x_trainscaled,y_train)
    print("The best training score is : ",'%.4f' %grid.score(x_trainscaled,y_train))
    print("The best test score is : ",'%.4f' %grid.score(x_testscaled,y_test))
    print(grid.best_params_)
    svcgp=grid.predict(x_testscaled)
    print(confusion_matrix(y_test,svcgp))
    x_poly=x_test.copy()
    grid5=grid
    x_poly['Predicted Value']=grid5.predict(x_testscaled)
    x_poly['Model']='Polynomial SVC'

    ### Decision Tree
    dt=DecisionTreeClassifier()
    value={"max_depth":range(1,20)}
    grid=GridSearchCV(dt,param_grid=value, scoring='recall')
    grid.fit(x_trainscaled,y_train)
    print("The best training score is : ",'%.4f' %grid.score(x_trainscaled,y_train))
    print("The best test score is : ",'%.4f' %grid.score(x_testscaled,y_test))
    print(grid.best_params_)
    dtp=grid.predict(x_testscaled)
    print(confusion_matrix(y_test,dtp))
    x_dec=x_test.copy()
    grid6=grid
    x_dec['Predicted Value']=grid6.predict(x_testscaled)
    x_dec['Model']='Decision Tree'

    ### Random Forest
    rf = RandomForestClassifier(random_state = 0)
    value={"n_estimators":range(1,40)}
    grid=GridSearchCV(rf,param_grid=value, scoring='recall')
    grid.fit(x_trainscaled,y_train)
    print("The best training score is : ",'%.4f' %grid.score(x_trainscaled,y_train))
    print("The best test score is : ",'%.4f' %grid.score(x_testscaled,y_test))
    print(grid.best_params_)
    rfp=grid.predict(x_testscaled)
    print(confusion_matrix(y_test,rfp))
    x_rand=x_test.copy()
    v=grid.best_params_
    grid7=RandomForestClassifier(random_state = 0,n_estimators=v['n_estimators'])
    grid7.fit(x_trainscaled,y_train)
    x_rand['Predicted Value']=grid7.predict(x_testscaled)
    x_rand['Model']='Random Forest'
    
    # Feature Importance
    importance = grid7.feature_importances_
    col=df.columns
    imp=[]
    for i in range(len(importance)):
        imp.append([col[i], importance[i]])
    impdf=pd.DataFrame(imp)
    impdf.columns=(['Feature','Importance'])
    print(impdf)
    impdf['Train']=mlmodel

    ### Voting
    vote = VotingClassifier(estimators=[('knn', grid1), 
                                        ('log', grid2), 
                                        ('svc', grid3),
                                        ('rbf', grid4),
                                        ('pol', grid5),
                                        ('des', grid6),
                                        ('ran', grid7),],    voting='hard')
    vote.fit(x_trainscaled,y_train)
    print("Training :",vote.score(x_trainscaled,y_train))
    print("Testing  :",vote.score(x_testscaled,y_test))
    pre=vote.predict(x_testscaled)
    print(confusion_matrix(y_test,pre))
    x_vote=x_test.copy()
    x_vote['Predicted Value']=vote.predict(x_testscaled)
    x_vote['Model']='Voting Classifier'

    ### NN
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=columns, activation='tanh'))
    model.add(Dense(40, activation='tanh'))
    model.add(Dense(20, activation='tanh'))
    model.add(Dense(10, activation='tanh'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(x_trainscaled, y_train, epochs=250, batch_size=16)
    # evaluate the model
    scores = model.evaluate(x_testscaled, y_test)
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print(confusion_matrix(y_test,model.predict(x_testscaled).round()))
    x_nn=x_test.copy()
    x_nn['probability']=model.predict(x_testscaled)
    x_nn['Model']='N-N'
    x_nn['Predicted Value']=round(x_nn['probability']).astype(np.int)

    ### Merge
    output=pd.concat([x_knn,x_log,x_lsvc,x_rbf,x_poly,x_dec,x_rand,x_vote], axis=0)
    NNoutput=x_nn.copy()
    output['Train']=mlmodel
    NNoutput['Train']=mlmodel
    
    ### Database
    if j==0:
        output.to_sql(name='PD', con=engine, if_exists='replace', index=True)
        NNoutput.to_sql(name='NNPD', con=engine, if_exists='replace', index=True)
        impdf.to_sql(name='FI', con=engine, if_exists='replace', index=True)
    else :
        output.to_sql(name='PD', con=engine, if_exists='append', index=True)
        NNoutput.to_sql(name='NNPD', con=engine, if_exists='append', index=True)
        impdf.to_sql(name='FI', con=engine, if_exists='append', index=True)
    
    print("Loop :",j)
    j=j+1
    

NameError: name 'df1' is not defined